In [1]:
!pip install tensorflow==2.4.3

     |████████████████████████████████| 394.5 MB 389 bytes/s 
     |████████████████████████████████| 462 kB 48.0 MB/s 
     |████████████████████████████████| 3.8 MB 33.5 MB/s 
     |████████████████████████████████| 2.9 MB 33.4 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68723 sha256=61ebe818192a857cf21d751b98f3cb9c26a84b408abddc157c9a076f56c86178
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
     

In [2]:
import tensorflow as tf
tf.__version__

'2.4.3'

In [3]:
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset/amazon_reviews_us_Shoes_v1_00.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 54101: expected 15 fields, saw 22\nSkipping line 55857: expected 15 fields, saw 22\nSkipping line 60448: expected 15 fields, saw 22\n'
b'Skipping line 76918: expected 15 fields, saw 22\nSkipping line 87925: expected 15 fields, saw 22\nSkipping line 88500: expected 15 fields, saw 22\nSkipping line 114276: expected 15 fields, saw 22\nSkipping line 128751: expected 15 fields, saw 22\n'
b'Skipping line 136095: expected 15 fields, saw 22\nSkipping line 140007: expected 15 fields, saw 22\nSkipping line 177148: expected 15 fields, saw 22\nSkipping line 180087: expected 15 fields, saw 22\nSkipping line 183010: expected 15 fields, saw 22\nSkipping line 183949: expected 15 fields, saw 22\nSkipping line 192879: expected 15 fields, saw 22\n'
b'Skipping line 223261: expected 15 fields, saw 22\nSkipping line 240588: expected 15 fields, saw 22\nSkipping line 247955: expected 15 fields, saw 22\nSkipping line 249336: expected 15 fields, saw 22\nSkipping line 259363: expected 15 fields, 

In [6]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,18069663,R3P2HIOQCIN5ZU,B000XB31C0,265024781,Minnetonka Men's Double Deerskin Softsole Mocc...,Shoes,1,0,0,N,Y,.,Do not buy: really didn't start to wear them u...,2015-08-31
1,US,16251825,R12VVR0WH5Q24V,B00CFYZH5W,259035853,Teva Men's Pajaro Flip-Flop,Shoes,5,0,0,N,Y,super flip flop,provides great cushion as well as archsupport,2015-08-31
2,US,20381037,RNCCKB6TV5EEF,B00S8JNN3Q,666066660,Anne Klein Perfect Pair Wristlet,Shoes,4,0,0,N,Y,Great clutch purse!,It's perfect if you need something small for c...,2015-08-31
3,US,108364,R2NZXYIVCGB13W,B00XFBPOQG,448483263,adidas Men's 10K Lifestyle Runner Sneaker,Shoes,5,0,6,N,Y,Badass,Getting what u see,2015-08-31
4,US,45449350,R2EQ1TG9IT3OEQ,B00SW64Y9W,7853171,OverBling Sneakers for Men Casual Men Shoes Ge...,Shoes,3,0,0,N,Y,Three Stars,small,2015-08-31


In [7]:
df['sentiments'] = df.star_rating.apply(lambda x: 0 if x in [1, 2, 3] else 1)

In [8]:
df[df['sentiments']==1]['review_id'][1]

'R12VVR0WH5Q24V'

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow import keras
import pickle

In [10]:
tokenizer = Tokenizer(oov_token="<OOV>")

In [11]:
split = round(len(df)*0.8)
train_reviews = df['review_id'][:split]
train_label = df['sentiments'][:split]
test_reviews = df['review_id'][split:]
test_label = df['sentiments'][split:]

In [12]:
train_reviews

0          R3P2HIOQCIN5ZU
1          R12VVR0WH5Q24V
2           RNCCKB6TV5EEF
3          R2NZXYIVCGB13W
4          R2EQ1TG9IT3OEQ
                ...      
3487051     RF1IU5H2OBAS8
3487052    R3UJB5HMVZMZH4
3487053    R2SFTPO5NUJM02
3487054      RZUST4DW16CA
3487055    R3Q8VDQ14AEFE3
Name: review_id, Length: 3487056, dtype: object

In [13]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for column in train_reviews:
    training_sentences.append(str(column))
for column in train_label:
    training_labels.append(column)
for column in test_reviews:
    testing_sentences.append(str(column))
for column in test_label:
    testing_labels.append(column)

In [14]:
training_sentences

['R3P2HIOQCIN5ZU',
 'R12VVR0WH5Q24V',
 'RNCCKB6TV5EEF',
 'R2NZXYIVCGB13W',
 'R2EQ1TG9IT3OEQ',
 'R1WXA9JSC2H1U1',
 'R12ENYLFGGNWRV',
 'R2R07E5PNXEUO5',
 'R27BA52AKWMWN3',
 'RLF8DOID2KD5O',
 'R369CEXHXHC6NQ',
 'R171PPIJXFONVI',
 'R2JDNM8F2P06FU',
 'R2W977FO4M97XT',
 'R3AM24QPLI28UY',
 'REDVXSFYVNT5T',
 'R14AIIK7D6ENDZ',
 'R3B1NURKMCVAL1',
 'R14Q1GZGV10IMX',
 'R9BHBB06QD6TM',
 'R3HVI80IL8TBSD',
 'R32I1BMKWJ463H',
 'RW4TDN6I1NTAN',
 'R132P1Y7J9WJAJ',
 'R1KJ8NGASTUQ80',
 'R2IVR3NBCUR6UA',
 'R321Q1D3ZX1PV9',
 'R38C58B3W36A2P',
 'R21XKRB2E1QBDH',
 'R231YXKUPKEI7B',
 'R3E047NHC63QAX',
 'R3N5SFHHOKDS4A',
 'R3VTIAWM3Y1435',
 'RFN9JY10X50F3',
 'R3ITTCICRV7972',
 'R2JW9KWGF7N7X8',
 'R1SEH88OC9KBZ',
 'R1XBT5YPGS0E27',
 'R2QS32P5QL3H8U',
 'R13749PRFL9JKT',
 'R1HD45TKWOB58H',
 'R2EQMSUZ9TX14Q',
 'R3OCNKPZWTVJUG',
 'R1RZSE28IVA1VK',
 'RE9KJGO100EDW',
 'R1VIUFLYMOMUUQ',
 'R361Y7H5NSD0AQ',
 'RDDUBX8JJKMX2',
 'R3PY1J43MATN5',
 'R31DGFI85N1HPA',
 'R3BGHM37O8TRS2',
 'R3EQ1M3MYK3KOI',
 'R2DIF0QUXSMOV0',
 'R

In [15]:
training_sentences[0]

'R3P2HIOQCIN5ZU'

In [16]:
training_labels[0]

0

In [17]:
vocab_size = 40000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [18]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [19]:
tokenizer

In [20]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [21]:
sequences

[[2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [78],
 [79],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [86],
 [87],
 [88],
 [89],
 [90],
 [91],
 [92],
 [93],
 [94],
 [95],
 [96],
 [97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104],
 [105],
 [106],
 [107],
 [108],
 [109],
 [110],
 [111],
 [112],
 [113],
 [114],
 [115],
 [116],
 [117],
 [118],
 [119],
 [120],
 [121],
 [122],
 [123],
 [124],
 [125],
 [126],
 [127],
 [128],
 [129],
 [130],
 [131],
 [132],
 [133],
 [134],
 [135],
 [136],
 [137],
 [138],
 [139],
 [

In [22]:
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [23]:
padded

array([[0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 4],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [24]:
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [25]:
type(training_labels)

list

In [26]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [27]:
training_labels_final

array([0, 1, 1, ..., 1, 1, 1])

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
num_epochs = 20
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/20
108971/108971 [==============================] - 1023s 9ms/step - loss: 0.5011 - accuracy: 0.8003 - val_loss: 0.5133 - val_accuracy: 0.7909
Epoch 2/20
108971/108971 [==============================] - 1015s 9ms/step - loss: 0.4992 - accuracy: 0.8010 - val_loss: 0.5132 - val_accuracy: 0.7909
Epoch 3/20
108971/108971 [==============================] - 1023s 9ms/step - loss: 0.4993 - accuracy: 0.8008 - val_loss: 0.5131 - val_accuracy: 0.7909
Epoch 4/20
 51069/108971 [=============>................] - ETA: 8:25 - loss: 0.4981 - accuracy: 0.8017

In [ ]:
import matplotlib.pyplot as plt
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
sample_text = [df[df['sentiments']==1]['review_id'][1]]
sample_text

In [ ]:
sample_sequences = tokenizer.texts_to_sequences(sample_text)

In [ ]:
sample_padded = pad_sequences(sample_sequences, maxlen=max_length, truncating=trunc_type)

In [ ]:
predictions = model.predict(sample_padded)
predictions

In [ ]:
if predictions[0][0] > 0.5:
  print('Review Positive')
else:
  print('Review Negative')

In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.save('./test_model', save_format='tf')

In [ ]:
!zip -r 'test_model.zip' 'test_model'